In [1]:
import pandas as pd
from obd import bts_parquet_path, bts_item_context_path, train_model, preprocess, preprocess_item_context, downcast, cost_name
from vowpalwabbit import Workspace

In [2]:
df = pd.read_parquet(bts_parquet_path)
df, x_encoder = preprocess(df)
df = downcast(df)
df.to_parquet("processed.parquet")
df.head()

,position,user_feature_0,user_feature_1,user_feature_2,user_feature_3,user-item_affinity_0,user-item_affinity_1,user-item_affinity_2,user-item_affinity_3,user-item_affinity_4,...,user-item_affinity_75,user-item_affinity_76,user-item_affinity_77,user-item_affinity_78,user-item_affinity_79,hour,day,item_id,click,propensity_score
0,1,2,4,0,9,0,0,0,0,0,...,0,0,0,0,0,0,0,18,1,0.061300
1,0,2,4,0,9,0,0,0,0,0,...,0,0,0,0,0,0,0,61,1,0.104925
2,2,2,4,0,9,0,0,0,0,0,...,0,0,0,0,0,0,0,51,1,0.048620
3,0,2,4,0,9,0,0,0,0,0,...,0,0,0,0,0,0,0,52,1,0.042980
4,1,2,4,0,9,0,0,0,0,0,...,0,0,0,0,0,0,0,35,1,0.005890


In [3]:
df = pd.read_parquet("processed.parquet")
df.head()

,position,user_feature_0,user_feature_1,user_feature_2,user_feature_3,user-item_affinity_0,user-item_affinity_1,user-item_affinity_2,user-item_affinity_3,user-item_affinity_4,...,user-item_affinity_75,user-item_affinity_76,user-item_affinity_77,user-item_affinity_78,user-item_affinity_79,hour,day,item_id,click,propensity_score
0,1,2,4,0,9,0,0,0,0,0,...,0,0,0,0,0,0,0,18,1,0.061300
1,0,2,4,0,9,0,0,0,0,0,...,0,0,0,0,0,0,0,61,1,0.104925
2,2,2,4,0,9,0,0,0,0,0,...,0,0,0,0,0,0,0,51,1,0.048620
3,0,2,4,0,9,0,0,0,0,0,...,0,0,0,0,0,0,0,52,1,0.042980
4,1,2,4,0,9,0,0,0,0,0,...,0,0,0,0,0,0,0,35,1,0.005890


In [4]:
action_context = pd.read_csv(bts_item_context_path)
action_context = action_context.drop(["Unnamed: 0"], axis=1)
action_context, ac_encoder = preprocess_item_context(action_context)
action_context.head()

,item_id,item_feature_0,item_feature_1,item_feature_2,item_feature_3
0,0,13.0,5.0,2.0,3.0
1,1,11.0,7.0,2.0,3.0
2,2,31.0,7.0,8.0,5.0
3,3,12.0,11.0,15.0,2.0
4,4,36.0,1.0,20.0,5.0


In [5]:
models = {"ips": Workspace("--cb_adf --cb_type ips", enable_logging=True), "dr": Workspace("--cb_adf --cb_type dr", enable_logging=True)}
losses = dict.fromkeys(["ips", "dr"])
for _, model in models.items():
    train_model(model, df, action_context)

100%|██████████| 12357200/12357200 [4:17:56<00:00, 798.47it/s] 


In [6]:
for k, model in models.items():    
    model.finish()
    for stat in model.get_log():
        if stat.startswith("average loss"):
            losses[k] = float(stat.split(" ")[3])
bts_loss = sum(df[cost_name])/len(df)
losses["ips"], losses["dr"], bts_loss

(0.817663, 0.945803, 0.9950467743501764)

In [7]:
ctrs = 1 - losses["ips"], 1 - losses["dr"], 1 - bts_loss
ctrs

(0.18233699999999997, 0.05419700000000005, 0.004953225649823589)